In [12]:
import pandas as pd
import numpy as np
import sklearn as sk
import warnings
warnings.filterwarnings("ignore")
import os, sys
os.chdir(r"C:\Users\ACER\Desktop\Terragon recruitment")

In [13]:
train_set = pd.read_csv(r"Train.csv")
test_set = pd.read_csv(r"Test.csv")

In [14]:
train_sample = train_set
train_sample.head()

,customer_id,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,deposit
0,RGD002844,31,management,married,tertiary,no,81,yes,no,cellular,29,oct,164,2,-1,0,unknown,1
1,RGD003806,62,retired,married,secondary,no,569,no,no,cellular,3,aug,187,2,180,6,success,1
2,RGD008310,35,technician,married,tertiary,no,432,no,no,cellular,12,aug,104,8,-1,0,unknown,0
3,RGD001840,43,management,married,tertiary,no,1429,yes,no,cellular,7,may,1030,1,169,3,success,1
4,RGD005881,29,blue-collar,married,primary,no,25,yes,no,unknown,4,jun,188,2,-1,0,unknown,0


In [15]:
check_train = pd.get_dummies(train_sample, columns = ['job', 'marital', 'education','default', 'housing','loan', 'contact','month','poutcome'])

In [16]:
preprocess_train = check_train
first_preprocess_train =preprocess_train.drop(['customer_id', 'deposit'], axis = 1)

In [17]:
import sklearn as sk
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

from sklearn.cross_validation import train_test_split

In [18]:
x = first_preprocess_train
y = check_train['deposit']

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.30, random_state =80)

In [19]:
scale = MinMaxScaler()
std_scale = StandardScaler()
x_train_std = std_scale.fit_transform(x_train)
x_test_std = std_scale.fit_transform(x_test)

x_train_scale = scale.fit_transform(x_train_std)
x_test_scale = scale.fit_transform(x_test_std)

In [20]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn import cross_validation, metrics   #Additional scklearn functions
from sklearn.grid_search import GridSearchCV   #Perforing grid search

gbrt = GradientBoostingClassifier(random_state = 32)
def modelfit(alg, x_train_scale, y_train, performCV=True, printFeatureImportance=False, cv_folds=5):
    #Fit the algorithm on the data
    alg.fit(x_train_scale, y_train)
        
    #Predict training set:
    dtrain_predictions = alg.predict(x_train_scale)
    dtrain_predprob = alg.predict_proba(x_train_scale)[:,1]
    
    #Perform cross-validation:
    if performCV:
        cv_score = cross_validation.cross_val_score(alg,x_train_scale, y_train, cv=cv_folds, scoring='roc_auc')
    
    #Print model report:
    print ("\nModel Report")
    print ("Accuracy : %.4g" % metrics.accuracy_score(y_train, dtrain_predictions))
    print ("AUC Score (Train): %f" % metrics.roc_auc_score(y_train, dtrain_predprob))
    
    if performCV:
        print ("CV Score : Mean - %.7g | Std - %.7g | Min - %.7g | Max - %.7g" % (np.mean(cv_score),np.std(cv_score),np.min(cv_score),np.max(cv_score)))
        
    #Print Feature Importance:
    if printFeatureImportance:
        feat_imp = pd.Series(alg.feature_importances_,y_train).sort_values(ascending=False)
        feat_imp.plot(kind='bar', title='Feature Importances')
        plt.ylabel('Feature Importance Score')
        plt.show()
modelfit(gbrt, x_train_scale, y_train)


Model Report
Accuracy : 0.8637
AUC Score (Train): 0.938255
CV Score : Mean - 0.9187887 | Std - 0.00769786 | Min - 0.9090413 | Max - 0.9310501


In [41]:
#Choose all predictors except target & IDcols

predictors = [x for x in train_set.columns if x not in ['deposit', 'customer_id']]
param_test1 = {'n_estimators':[260, 270,280,290,300]}
gsearch1 = GridSearchCV(estimator = GradientBoostingClassifier(learning_rate=0.1, min_samples_split=500,min_samples_leaf=50,max_depth=8,max_features='sqrt',subsample=0.8,random_state=32), 
param_grid = param_test1, scoring='roc_auc',n_jobs=4,iid=False, cv=5)
gsearch1.fit(x_train_scale,y_train)

GridSearchCV(cv=5, error_score='raise',
       estimator=GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=8,
              max_features='sqrt', max_leaf_nodes=None,
              min_impurity_split=1e-07, min_samples_leaf=50,
              min_samples_split=500, min_weight_fraction_leaf=0.0,
              n_estimators=100, presort='auto', random_state=32,
              subsample=0.8, verbose=0, warm_start=False),
       fit_params={}, iid=False, n_jobs=4,
       param_grid={'n_estimators': [260, 270, 280, 290, 300]},
       pre_dispatch='2*n_jobs', refit=True, scoring='roc_auc', verbose=0)

In [42]:
gsearch1.grid_scores_

[mean: 0.92534, std: 0.00946, params: {'n_estimators': 260},
 mean: 0.92556, std: 0.00938, params: {'n_estimators': 270},
 mean: 0.92552, std: 0.00927, params: {'n_estimators': 280},
 mean: 0.92566, std: 0.00936, params: {'n_estimators': 290},
 mean: 0.92573, std: 0.00935, params: {'n_estimators': 300}]

In [43]:
dtrain_predprob = gsearch1.predict_proba(x_train_scale)[:,1] 
print ("AUC Score (Train): %f" % metrics.roc_auc_score(y_train, dtrain_predprob))

AUC Score (Train): 0.971938


In [44]:
gsearch1.best_params_

{'n_estimators': 300}

In [45]:
gsearch1.best_score_

0.925725122269155

## start below

In [52]:
param_test2 = {'max_depth':[2,4,6,8,10], 'min_samples_split':[100,200,300,400]}
gsearch2 = GridSearchCV(estimator = GradientBoostingClassifier(learning_rate=0.1, n_estimators=300, max_features='sqrt', subsample=0.8, random_state=32),
                        param_grid = param_test2, scoring='roc_auc',n_jobs=4,iid=False, cv=5)
gsearch2.fit(x_train_scale,y_train)
gsearch2.grid_scores_

[mean: 0.91777, std: 0.00807, params: {'max_depth': 2, 'min_samples_split': 100},
 mean: 0.91715, std: 0.00797, params: {'max_depth': 2, 'min_samples_split': 200},
 mean: 0.91688, std: 0.00846, params: {'max_depth': 2, 'min_samples_split': 300},
 mean: 0.91536, std: 0.00957, params: {'max_depth': 2, 'min_samples_split': 400},
 mean: 0.92740, std: 0.00796, params: {'max_depth': 4, 'min_samples_split': 100},
 mean: 0.92739, std: 0.00789, params: {'max_depth': 4, 'min_samples_split': 200},
 mean: 0.92608, std: 0.00834, params: {'max_depth': 4, 'min_samples_split': 300},
 mean: 0.92540, std: 0.00863, params: {'max_depth': 4, 'min_samples_split': 400},
 mean: 0.93038, std: 0.00785, params: {'max_depth': 6, 'min_samples_split': 100},
 mean: 0.92951, std: 0.00817, params: {'max_depth': 6, 'min_samples_split': 200},
 mean: 0.92880, std: 0.00879, params: {'max_depth': 6, 'min_samples_split': 300},
 mean: 0.92768, std: 0.00977, params: {'max_depth': 6, 'min_samples_split': 400},
 mean: 0.92903, 

In [53]:
dtrain_predprob = gsearch2.predict_proba(x_train_scale)[:,1] 
print ("AUC Score (Train): %f" % metrics.roc_auc_score(y_train, dtrain_predprob))

AUC Score (Train): 0.988163


In [54]:
gsearch2.best_params_ 

{'max_depth': 6, 'min_samples_split': 100}

In [55]:
gsearch2.best_score_

0.9303827314366192

In [59]:
param_test3 = {'min_samples_leaf':[10,20,30]}
gsearch3 = GridSearchCV(estimator = GradientBoostingClassifier(learning_rate=0.1,min_samples_split = 100, n_estimators=300,max_depth=6,max_features='sqrt', subsample=0.8, random_state=32), 
                    param_grid = param_test3, scoring='roc_auc',n_jobs=4,iid=False, cv=5)
gsearch3.fit(x_train_scale,y_train)
gsearch3.grid_scores_

[mean: 0.92866, std: 0.00888, params: {'min_samples_leaf': 10},
 mean: 0.92877, std: 0.00873, params: {'min_samples_leaf': 20},
 mean: 0.92837, std: 0.00821, params: {'min_samples_leaf': 30}]

In [60]:
dtrain_predprob = gsearch3.predict_proba(x_train_scale)[:,1] 
print ("AUC Score (Train): %f" % metrics.roc_auc_score(y_train, dtrain_predprob))

AUC Score (Train): 0.984700


In [61]:
gsearch3.best_params_

{'min_samples_leaf': 20}

In [62]:
gsearch3.best_score_

0.9287740536569421

In [36]:
modelfit(gsearch3.best_estimator_, x_train_scale, y_train)


Model Report
Accuracy : 0.8915
AUC Score (Train): 0.959910
CV Score : Mean - 0.922666 | Std - 0.003433451 | Min - 0.9188114 | Max - 0.928373


In [63]:
param_test4 = {'max_features':[16,18,20]}
gsearch4 = GridSearchCV(estimator = GradientBoostingClassifier(learning_rate=0.1, n_estimators=300,max_depth=6, min_samples_split=100, min_samples_leaf=20, subsample=0.8, random_state=32),
                        param_grid = param_test4, scoring='roc_auc',n_jobs=4,iid=False, cv=5)
gsearch4.fit(x_train_scale,y_train)
gsearch4.grid_scores_

[mean: 0.92767, std: 0.00833, params: {'max_features': 16},
 mean: 0.92910, std: 0.00787, params: {'max_features': 18},
 mean: 0.92755, std: 0.00740, params: {'max_features': 20}]

In [66]:
dtrain_predprob = gsearch4.predict_proba(x_train_scale)[:,1] 
print ("AUC Score (Train): %f" % metrics.roc_auc_score(y_train, dtrain_predprob))

AUC Score (Train): 0.991748


In [67]:
gsearch4.best_params_

{'max_features': 18}

In [68]:
gsearch4.best_score_

0.9291018869665404

In [72]:
param_test5 = {'subsample':[0.84,0.86,0.88,0.9]}
gsearch5 = GridSearchCV(estimator = GradientBoostingClassifier(learning_rate=0.1, n_estimators=300,max_depth=6,min_samples_split=100, min_samples_leaf=20, random_state=32,max_features=18),
                        param_grid = param_test5, scoring='roc_auc',n_jobs=4,iid=False, cv=5)
gsearch5.fit(x_train_scale,y_train)
gsearch5.grid_scores_

[mean: 0.92821, std: 0.00775, params: {'subsample': 0.84},
 mean: 0.92811, std: 0.00724, params: {'subsample': 0.86},
 mean: 0.92889, std: 0.00783, params: {'subsample': 0.88},
 mean: 0.92942, std: 0.00747, params: {'subsample': 0.9}]

In [73]:
dtrain_predprob = gsearch5.predict_proba(x_train_scale)[:,1] 
print ("AUC Score (Train): %f" % metrics.roc_auc_score(y_train, dtrain_predprob))

AUC Score (Train): 0.992441


In [74]:
gsearch5.best_params_

{'subsample': 0.9}

In [75]:
gsearch5.best_score_

0.9294220315969592

In [76]:
predictors = [x for x in train_set.columns if x not in ['deposit', 'id_column']]
gbm_tuned_1 = GradientBoostingClassifier(learning_rate=0.05, n_estimators=300,max_depth=6, min_samples_split=100, min_samples_leaf =20,subsample = 0.9, random_state = 32, max_features = 18)
modelfit(gbm_tuned_1, x_train_scale,y_train)


Model Report
Accuracy : 0.917
AUC Score (Train): 0.977143
CV Score : Mean - 0.929112 | Std - 0.007140326 | Min - 0.9200552 | Max - 0.9401718


In [77]:
predictors = [x for x in train_set.columns if x not in ['deposit', 'id_column']]
gbm_tuned_3 = GradientBoostingClassifier(learning_rate=0.01, n_estimators=1500,max_depth=8, min_samples_split=100, min_samples_leaf=20, subsample=0.9, random_state=32, max_features=18,warm_start=True)
modelfit(gbm_tuned_3, x_train_scale, y_train, performCV=False)


Model Report
Accuracy : 0.9427
AUC Score (Train): 0.990311


In [78]:
predictors = [x for x in train_set.columns if x not in ['deposit', 'id_column']]
gbm_tuned_4 = GradientBoostingClassifier(learning_rate=0.005, n_estimators=3000,max_depth=8, min_samples_split=100, min_samples_leaf=20, subsample=0.9, random_state=32, max_features=18,warm_start=True)
modelfit(gbm_tuned_4, x_train_scale, y_train, performCV=False)


Model Report
Accuracy : 0.9426
AUC Score (Train): 0.990514


In [80]:
from sklearn.model_selection import cross_val_predict
confusion_value, precision, recall, f1 = np.array((2,2)), 0, 0, 0
y_train_predict = cross_val_predict(gbm_tuned_4, x_train_scale, y_train, cv = 5)
from sklearn.metrics import confusion_matrix
confusion_value = confusion_matrix(y_train, y_train_predict)
from sklearn.metrics import precision_score, recall_score, f1_score
precision = precision_score(y_train, y_train_predict)
recall = recall_score(y_train, y_train_predict)
f1 = f1_score(y_train, y_train_predict)
print (precision, recall, f1)

0.838489091501 0.87704359673 0.85733311137


In [81]:
from sklearn.model_selection import cross_val_predict
confusion_value, precision, recall, f1 = np.array((2,2)), 0, 0, 0
y_train_predict = cross_val_predict(gsearch5, x_train_scale, y_train, cv = 5)
from sklearn.metrics import confusion_matrix
confusion_value = confusion_matrix(y_train, y_train_predict)
from sklearn.metrics import precision_score, recall_score, f1_score
precision = precision_score(y_train, y_train_predict)
recall = recall_score(y_train, y_train_predict)
f1 = f1_score(y_train, y_train_predict)
print (precision, recall, f1)

0.83835078534 0.872615803815 0.855140186916


In [82]:
check_test = pd.get_dummies(test_set, columns = ['job', 'marital', 'education','default', 'housing','loan', 'contact','month','poutcome'])

In [83]:
preprocess_test = check_test
first_preprocess_test =preprocess_test.drop('customer_id', axis = 1)

In [84]:
first_preprocess_scale = scale.fit_transform(first_preprocess_test)
first_preprocess_scaled = std_scale.fit_transform(first_preprocess_scale)
first_preprocess_scaled

array([[-0.26097789, -0.46829201, -0.9018059 , ..., -0.21990181,
        -0.3363934 ,  0.59648841],
       [-0.85485552,  0.07397407, -1.01886977, ..., -0.21990181,
        -0.3363934 , -1.67647851],
       [ 1.43581533, -0.40877054, -0.55061431, ..., -0.21990181,
        -0.3363934 ,  0.59648841],
       ..., 
       [-1.02453484, -0.14840979,  0.85415205, ..., -0.21990181,
        -0.3363934 , -1.67647851],
       [ 1.43581533, -0.67314808, -1.25299749, ..., -0.21990181,
         2.97270994, -1.67647851],
       [-0.6851762 , -0.2831547 , -0.19942272, ...,  4.5474841 ,
        -0.3363934 , -1.67647851]])

In [89]:
gbm_tuned_result =gbm_tuned_4.predict(first_preprocess_scaled)
dict_gbm_tuned = {'customer_id': test_set['customer_id'], 'deposit': gbm_tuned_result}
to_pd_gbm_tuned = pd.DataFrame(dict_gbm_tuned)
to_pd_gbm_tuned.to_csv(r"C:\Users\ACER\Desktop\Terragon recruitment\gbm_tuned_4.csv", index = False)

In [90]:
gsearch_result =gsearch5.predict(first_preprocess_scaled)
gsearch_tuned = {'customer_id': test_set['customer_id'], 'deposit': gsearch_result}
to_pd_gsearch_tuned = pd.DataFrame(gsearch_tuned)
to_pd_gsearch_tuned.to_csv(r"C:\Users\ACER\Desktop\Terragon recruitment\gsearch.csv", index =False )

In [88]:
help (to_pd_gsearch_tuned.to_csv)

Help on method to_csv in module pandas.core.frame:

to_csv(path_or_buf=None, sep=',', na_rep='', float_format=None, columns=None, header=True, index=True, index_label=None, mode='w', encoding=None, compression=None, quoting=None, quotechar='"', line_terminator='\n', chunksize=None, tupleize_cols=False, date_format=None, doublequote=True, escapechar=None, decimal='.') method of pandas.core.frame.DataFrame instance
    Write DataFrame to a comma-separated values (csv) file
    
    Parameters
    ----------
    path_or_buf : string or file handle, default None
        File path or object, if None is provided the result is returned as
        a string.
    sep : character, default ','
        Field delimiter for the output file.
    na_rep : string, default ''
        Missing data representation
    float_format : string, default None
        Format string for floating point numbers
    columns : sequence, optional
        Columns to write
    header : boolean or list of string, default T

In [1]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()

In [10]:
from sklearn import cross_validation, metrics   #Additional scklearn functions
from sklearn.grid_search import GridSearchCV   #Perforing grid search

lr.get_params()

{'C': 1.0,
 'class_weight': None,
 'dual': False,
 'fit_intercept': True,
 'intercept_scaling': 1,
 'max_iter': 100,
 'multi_class': 'ovr',
 'n_jobs': 1,
 'penalty': 'l2',
 'random_state': None,
 'solver': 'liblinear',
 'tol': 0.0001,
 'verbose': 0,
 'warm_start': False}

In [91]:
dtrain_predprob = gsearch5.predict_proba(x_train_scale)[:,1] 
print ("AUC Score (Train): %f" % metrics.roc_auc_score(y_train, dtrain_predprob))

AUC Score (Train): 0.992441


In [92]:
dtrain_predprob = gbm_tuned_4.predict_proba(x_train_scale)[:,1] 
print ("AUC Score (Train): %f" % metrics.roc_auc_score(y_train, dtrain_predprob))

AUC Score (Train): 0.990514
